# Training an intents classification model

Define constants

In [1]:
from os import getcwd, path
import sys
import matplotlib.pyplot as plt
import numpy as np

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

DATA_UTILS = path.join(BASE_PATH, 'common/data_utils.py')
TRAIN_PATH = path.join(BASE_PATH, 'kc_data.json')
CLASSES_FILE = path.join(BASE_PATH, 'classes.json')

In [2]:
exec(open(DATA_UTILS).read())

Use functions from the utils to extract and preprocess the training data
Refer to `kc_data.json` for the sample data format
`get_data_pairs` is then used to parse data into a tuple of `([list_of_sentences], [list_of_labels])`

In [3]:
X_train, y_train = get_data_pairs(data_from_json(TRAIN_PATH))

In [4]:
import torch
torch.__version__

'0.4.1'

Start training the classification model and save

In [5]:
from text_classification.fast_text.model import FastTextWrapper
from text_classification.fast_text.train import FastTextLearner
from common.callbacks import PrintLoggerCallback

model = FastTextWrapper()
learner = FastTextLearner(
    model,
    optimizer_fn='adam'
#     optimizer_fn='sgd',
#     optimizer_kwargs={'lr': 1e-2, 'momentum': .9}
)

In [ ]:
learner.fit(
    training_data=(X_train, y_train), 
    epochs=100, 
    callbacks=[PrintLoggerCallback(log_every=5)]
)

3m 58s (- 75m 31s) (5 5%) - loss: 7.3131 - accuracy: 0.0004
8m 10s (- 73m 31s) (10 10%) - loss: 7.2986 - accuracy: 0.0000
12m 27s (- 70m 35s) (15 15%) - loss: 7.2694 - accuracy: 0.0004
16m 33s (- 66m 12s) (20 20%) - loss: 7.0779 - accuracy: 0.0047
20m 26s (- 61m 18s) (25 25%) - loss: 6.3776 - accuracy: 0.0146
24m 26s (- 57m 2s) (30 30%) - loss: 5.5605 - accuracy: 0.0550
27m 59s (- 51m 59s) (35 35%) - loss: 4.7425 - accuracy: 0.1226
31m 46s (- 47m 40s) (40 40%) - loss: 3.9918 - accuracy: 0.2061
35m 45s (- 43m 42s) (45 45%) - loss: 3.3124 - accuracy: 0.3014
39m 44s (- 39m 44s) (50 50%) - loss: 2.7037 - accuracy: 0.4050
43m 38s (- 35m 42s) (55 55%) - loss: 2.1830 - accuracy: 0.5024
47m 41s (- 31m 47s) (60 60%) - loss: 1.7432 - accuracy: 0.5838
51m 52s (- 27m 55s) (65 65%) - loss: 1.3926 - accuracy: 0.6576
56m 1s (- 24m 0s) (70 70%) - loss: 1.1112 - accuracy: 0.7242
59m 56s (- 19m 58s) (75 75%) - loss: 0.8971 - accuracy: 0.7717
64m 1s (- 16m 0s) (80 80%) - loss: 0.7308 - accuracy: 0.8098


In [ ]:
model(['the earth is flat'])